## Tutorial: Create an OpenFlow Network and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

In [1]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project and site
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_skylake"
server_count=2
physical_network="physnet1"
of_controller_ip="192.5.87.215"
of_controller_port = "6653"
vswitch_name = username+"SDN"

#### Create a Lease

In [2]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network,of_controller_ip=of_controller_ip, of_controller_port=of_controller_port, vswitch_name=vswitch_name)
add_fip_reservation(reservation_list, count=2)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

{'status': 'PENDING',
 'user_id': 'f3a422ce5a4442e3a7a247ddfd98bfc3',
 'name': 'pruthLease',
 'end_date': '2020-07-24T16:37:00.000000',
 'reservations': [{'status': 'pending',
   'before_end': 'default',
   'lease_id': 'de0d65ce-f1e7-4d8b-a8e5-af4e7a60e935',
   'resource_id': '0d9cff25-1c75-489e-96ce-b4a893f5a972',
   'max': 2,
   'created_at': '2020-07-23 16:37:22',
   'min': 2,
   'updated_at': '2020-07-23 16:37:27',
   'missing_resources': False,
   'hypervisor_properties': '',
   'on_start': 'default',
   'resource_properties': '["==", "$node_type", "compute_skylake"]',
   'id': '073806a9-5be3-433c-bdd3-28d4f1e4f27a',
   'resource_type': 'physical:host',
   'resources_changed': False},
  {'status': 'pending',
   'lease_id': 'de0d65ce-f1e7-4d8b-a8e5-af4e7a60e935',
   'resource_id': 'fff628cb-33af-4fd8-a86d-6063d1ef1e2a',
   'created_at': '2020-07-23 16:37:29',
   'updated_at': '2020-07-23 16:37:31',
   'missing_resources': False,
   'id': '4b6bb8a0-0d6b-436c-9f97-897bbc887375',
   '

In [3]:
#Get the lease by name
lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

{
  "status": "ACTIVE",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-07-24T16:37:00.000000",
  "reservations": [
    {
      "status": "active",
      "before_end": "default",
      "lease_id": "de0d65ce-f1e7-4d8b-a8e5-af4e7a60e935",
      "resource_id": "0d9cff25-1c75-489e-96ce-b4a893f5a972",
      "max": 2,
      "created_at": "2020-07-23 16:37:22",
      "min": 2,
      "updated_at": "2020-07-23 16:38:11",
      "missing_resources": false,
      "hypervisor_properties": "",
      "on_start": "default",
      "resource_properties": "[\"==\", \"$node_type\", \"compute_skylake\"]",
      "id": "073806a9-5be3-433c-bdd3-28d4f1e4f27a",
      "resource_type": "physical:host",
      "resources_changed": false
    },
    {
      "status": "active",
      "lease_id": "de0d65ce-f1e7-4d8b-a8e5-af4e7a60e935",
      "resource_id": "fff628cb-33af-4fd8-a86d-6063d1ef1e2a",
      "created_at": "2020-07-23 16:37:29",
      "updated_at": "2020-07-23 16:38

#### Get the Reservations

In [4]:
#Get the lease by name
lease = get_lease_by_name(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

compute_reservation_id: 073806a9-5be3-433c-bdd3-28d4f1e4f27a
network_reservation_id: 4b6bb8a0-0d6b-436c-9f97-897bbc887375
floatingip_reservation_id: 85967402-0bc9-4476-800e-c3be79845923


#### Configute the Network

In [5]:
network = get_network_by_name(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

Network ID: b5d8695f-c153-4cf1-946d-1bc93ec0c7c8


In [6]:
subnet = add_subnet(subnet_name, network_name)
router = create_router(router_name, gw_network_name='public')
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': 'b5d8695f-c153-4cf1-946d-1bc93ec0c7c8',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '0d8621dc-5591-418a-bbcc-1438d1e6b1d1',
 'subnet_ids': ['0d8621dc-5591-418a-bbcc-1438d1e6b1d1'],
 'port_id': 'f03a507a-786d-4a5e-8b11-aa37b782051d',
 'id': '94ed8bf7-1d9d-410f-ae20-f5aab4aedda5'}

#### Start the Server

In [7]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [8]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 192.5.87.218


#### Delete Server

In [9]:
delete_server_by_name(server_name)

()

#### De-configure Network

In [10]:
try:
    result = detach_router_by_name(router_name=router_name, subnet_name=subnet_name)
except Exception as e:
    print("detach_router_by_name error:" + str(e))
    pass

try:
    result = delete_router_by_name(router_name)
except Exception as e:
    print("delete_router_by_name error: " + str(e))
    pass

try:
    result = delete_subnet_by_name(subnet_name)
except Exception as e:
    print("delete_subnet_by_name error: " + str(e))
    pass

try:
    result = delete_network_by_name(network_name)
except Exception as e:
    print("delete_network_by_name error: " + str(e))
    pass

#### Release Lease

In [11]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id de0d65ce-f1e7-4d8b-a8e5-af4e7a60e935
